In [1]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np
import keras2onnx
import onnxruntime
print(f"Tensoflow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
from models import endpoint_in_model
from losses import euclidean_distance_loss,endpoint_loss
from data_processing.data_loader import dataset_loader
from config import params

from time import time
import keract



os.environ['CUDA_VISIBLE_DEVICES']="1"
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Tensoflow Version: 2.3.0
Keras Version: 2.4.0
Num GPUs Available:  1


In [2]:
dataset_dir = '/netpool/work/gpu-3/users/malyalasa/New_folder/tf_records'
ds_loader = dataset_loader(tfrec_dir=dataset_dir, batch_size=32)
ds_train, ds_valid, ds_test = ds_loader.build_dataset()

Number of records, Train files:408, validation files:88, Test Files:87
Building Dataset.......

((TensorSpec(shape=(None, 1536, 1536), dtype=tf.int8, name=None),
  TensorSpec(shape=(None, None), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 25, 2), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 25, 2), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, None), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 25, 2), dtype=tf.float32, name=None),
  TensorSpec(shape=(None,), dtype=tf.string, name=None)),
 TensorSpec(shape=(None, 25, 2), dtype=tf.float32, name=None))


In [3]:
# load model wwith CustomMaskLayer
pp_model = tf.keras.models.load_model('/netpool/work/gpu-3/users/malyalasa/New_folder/wandb/latest-run/files/model-best.h5',compile=False,custom_objects={'CustomMaskLayer':endpoint_in_model.CustomMaskLayer})

pp_model.summary()
# Compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.02)
pp_model.compile(
        optimizer=opt, 
        loss=[euclidean_distance_loss,endpoint_loss],
        loss_weights=params.get("loss_weights"), metrics=params.get("metric"))

# evaluate
test_loss,test_accuracy = pp_model.evaluate(ds_test)


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1536, 1536,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 765, 765, 16) 800         input_1[0][0]                    
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 765, 765, 16) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 765, 765, 16) 64          re_lu[0][0]                      
_______________________________________________________________________________________

In [4]:
data = np.load('/netpool/work/gpu-3/users/malyalasa/New_folder/rosbag2numpy/dataset/sample_gridmap.npz')
input_sample = [data[f'{file}'] for file in data.files]

#print(len(input_sample))
pp_model.predict(input_sample)

array([[[  53.09721 , -848.1576  ],
        [ -23.843918, -975.952   ],
        [ -71.257385, -818.2038  ],
        [ -62.6431  , -826.1478  ],
        [ -31.1552  , -783.9839  ],
        [ 118.8456  , -765.6037  ],
        [ 269.877   , -511.67322 ],
        [ 276.40176 , -735.5734  ],
        [ 122.97023 , -423.0063  ],
        [ 164.49922 , -673.63354 ],
        [ 174.18005 , -681.5854  ],
        [ 250.53789 , -669.09406 ],
        [ 182.525   , -735.4793  ],
        [ 318.42258 , -589.0727  ],
        [ 337.98303 , -795.3982  ],
        [ 274.6278  , -823.6943  ],
        [ 200.51749 , -688.3573  ],
        [ 272.91992 , -715.28546 ],
        [ 199.0691  , -741.7165  ],
        [ 280.45856 , -635.4147  ],
        [ 353.952   , -556.41736 ],
        [ 206.26997 , -735.1424  ],
        [ 201.31079 , -799.86566 ],
        [ 195.50462 , -694.6135  ],
        [ 161.78111 , -722.82367 ]]], dtype=float32)

In [5]:
activations = keract.get_activations(pp_model, input_sample, auto_compile=True)

In [6]:
activations_dir = '/netpool/work/gpu-3/users/malyalasa/New_folder/rosbag2numpy/logging/act_newmodel'

keract.display_activations(activations, cmap=None, save=True, directory=activations_dir, data_format='channels_last', fig_size=(10, 10), reshape_1d_layers=False)


input_1 (1, 1536, 1536, 1) 
Grid_origin_res (1, 3) 
Left_boundary (1, 25, 2) 
Right_boundary (1, 25, 2) 
Car_loc (1, 3) 
Initial_path (1, 25, 2) 
conv2d (1, 765, 765, 16) 
re_lu (1, 765, 765, 16) 
batch_normalization (1, 765, 765, 16) 
average_pooling2d (1, 191, 191, 16) 
conv2d_1 (1, 191, 191, 8) 
conv2d_2 (1, 191, 191, 2) 
flatten (1, 72962) 
reshape (1, 50) 
reshape_1 (1, 50) 
reshape_2 (1, 50) 
concatenate (1, 6) 
concatenate_1 (1, 73118) 
dense (1, 16) 
batch_normalization_1 (1, 16) 
dense_1 (1, 50) 
add (1, 50) 
dense_2 (1, 50) 
reshape_3 (1, 25, 2) 
